In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null

import lightgbm as lgb
import datatable as dt
from datatable import f, by
import numpy as np 

## Load Model

In [ ]:
lgbm = lgb.Booster(model_file = '../input/model-building-with-cumulative-features/model.txt')

## Load datasets

In [ ]:
user_last_performance = dt.fread('../input/model-building-with-cumulative-features/user_stats.csv')
question_stats = dt.fread('../input/feature-engineering-datasets/questions_stats_fe.csv')
questions = dt.fread('../input/feature-engineering-datasets/questions_fe.csv')
questions.names={'question_id':'content_id'}
questions.key='content_id'
question_stats.names={'question_id':'content_id'}
question_stats.key='content_id'
user_last_performance.key='user_id'

In [ ]:
aggregation_dict = {}
aggregation_perf_dict = {}
for part in dt.unique(questions[:,'part']).to_list()[0]:
    for i in (0,1):
        aggregation_dict['part_'+str(part)+'_'+str(i)] = dt.sum((f.answered_correctly==0)&(f.part==part))
        aggregation_perf_dict['part_'+str(part)+'_'+str(i)] = dt.sum(f['part_'+str(part)+'_'+str(i)])
for part in dt.unique(questions[:,'kmean_cluster']).to_list()[0]:
    for i in (0,1):
        aggregation_dict['cluster_'+str(part)+'_'+str(i)] = dt.sum((f.answered_correctly==0)&(f.part==part))
        aggregation_perf_dict['cluster_'+str(part)+'_'+str(i)] = dt.sum(f['cluster_'+str(part)+'_'+str(i)])
aggregation_dict['timestamp'] = dt.max(f.timestamp)
aggregation_perf_dict['timestamp'] = dt.max(f.timestamp)

## Define functions

In [ ]:
#MODIFICARE DATASET CON DATATABLE MIGLIORANDO L'EFFICIENZA

def enrich_dataset(dataset, last_record):
    
    # RICEVO IL DATASET SU CUI APPLICARE I RISULTATI
    
    #AGGIUNGO GLI ATTRIBUTI PRECALCOLATI PER TUTENTE
    
    #AGGIUNGO GLI ATTRIBUTI DAI VARI DATASETS
    
    #RESTITUISCO DATASET ARRICCHITO
    
    data = dt.Frame(
        dataset[['content_type_id','row_id','user_id','content_id','timestamp','prior_question_elapsed_time']].fillna(-60*60*24)
    )
    content_type_id,row_id, user_id, content_id, timestamp, prior_question_elapsed_time = data.export_names()
    data = data[:,{'row_id':row_id, 
                   'user_id':user_id, 
                   'content_id':content_id, 
                   'timestamp':timestamp/(365*24*60*60*100), 
                   'prior_question_elapsed_time':prior_question_elapsed_time/(60*60*24),
                  'content_type_id':content_type_id}]
    data = data[:,:,dt.join(question_stats)]
    X = user_last_performance[:, dt.f[:].remove(dt.f['timestamp'])]
    X.key = 'user_id'
    data = data[:,:,dt.join(X)]
    
    del X

    for part in dt.unique(questions[:,'part']).to_list()[0]:
        col1 = 'part_'+str(part)+'_avg'
        col2 = 'part_'+str(part)+'_count'
        part_1 = 'part_'+str(part)+'_1'
        part_0 = 'part_'+str(part)+'_0'
        data[:,col1] = data[:,dt.f[part_1]/(dt.f[part_0]+dt.f[part_1])]
        data[:,col2] = data[:,dt.f[part_0] + dt.f[part_1]]
        del data[:, part_1]
        del data[:, part_0]
    for part in dt.unique(questions[:,'kmean_cluster']).to_list()[0]:
        col1 = 'cluster_'+str(part)+'_avg'
        col2 = 'cluster_'+str(part)+'_count'
        part_1 = 'cluster_'+str(part)+'_1'
        part_0 = 'cluster_'+str(part)+'_0'
        data[:,col1] = data[:,dt.f[part_1]/(dt.f[part_0]+dt.f[part_1])]
        data[:,col2] = data[:,dt.f[part_0] + dt.f[part_1]]
        del data[:, part_1]
        del data[:, part_0]
    return data.to_pandas().fillna(0.5)

In [ ]:
target_col = ['answered_correctly']
feature_col = [
 'timestamp',
 'prior_question_elapsed_time',
 'part_1_avg',
 #'part_1_count',
 'part_2_avg',
 #'part_2_count',
 'part_3_avg',
 #'part_3_count',
 'part_4_avg',
 #'part_4_count',
 'part_5_avg',
 #'part_5_count',
 'part_6_avg',
 #'part_6_count',
 'part_7_avg',
 #'part_7_count',
 'cluster_0_avg',
 #'cluster_0_count',
 'cluster_1_avg',
 #'cluster_1_count',
 'cluster_2_avg',
 #'cluster_2_count',
 'cluster_3_avg',
 #'cluster_3_count',
 'cluster_4_avg',
 #'cluster_4_count',
    'cluster_5_avg',
 #'cluster_5_count',
 'cluster_6_avg',
 #'cluster_6_count',
 'bundle_mean_answered_correctly',
 'bundle_std_answered_correctly',
 'bundle_perc_students',
 #'bundle_times',
 'part_mean_answered_correctly',
 'part_std_answered_correctly',
 'part_perc_students',
 #'part_times',
 'cluster_mean_answered_correctly',
 'cluster_std_answered_correctly',
 'cluster_perc_students'
# 'cluster_times'
]

In [ ]:
import riiideducation

env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
def get_last_performance(last_df, user_last):
    from datatable import f, by 
    data = dt.Frame(last_df.loc[last_df['content_type_id']==0,['timestamp','user_id','row_id','answered_correctly','content_id']])
    data.key = 'row_id'
    X = data[:,:,dt.join(questions)]
    del X[:,['content_id','bundle_id','tags']]
    X.key = 'row_id'
    data = X[:,aggregation_dict,by('user_id')]
    data.rbind(user_last_performance)
    X = data[:,aggregation_perf_dict,by('user_id')]    
    return X

In [ ]:
d = {'row_id':'int64','answered_correctly':'float64'}

import gc

last_df = pd.DataFrame()

for (test_df,current_prediction_df) in iter_test:
    
    gc.collect()
    
    if last_df.shape[0]>0:
        last_df['answered_correctly'] = np.array(eval(test_df['prior_group_answers_correct'].iloc[0]))
        #%time
        X = get_last_performance(last_df, user_last_performance)
        user_last_performance = X
        del X
    data = enrich_dataset(test_df, user_last_performance)
    data = data[data.content_type_id == 0]
    
    assert len(data)==len(current_prediction_df)
    
    current_prediction_df.answered_correctly = lgbm.predict(data[feature_col].fillna(-1)).ravel()
    del data
    
    env.predict(current_prediction_df.fillna(0.5).astype(d))
    last_df = test_df.copy(deep=True)